<a href="https://colab.research.google.com/github/camligorkem/cs-260c-project/blob/main/model_parameter_tuning/CS_260_Node_Classification_GNN_MLP_param_tuning_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

     |████████████████████████████████| 7.9 MB 11.5 MB/s 
     |████████████████████████████████| 3.5 MB 10.4 MB/s 


In [2]:
import torch 
import numpy as np
import math


from torch_geometric.utils import degree
import torch_geometric
import torch_geometric.utils as tg_utils

In [3]:
import pandas as pd

In [ ]:
!rm -r data

rm: cannot remove 'data': No such file or directory


In [4]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Processing...
Done!


In [5]:
!pip install class-resolver

from torch_geometric.nn import MLP, GCN, GraphSAGE, GAT
from class_resolver import ClassResolver

In [7]:
# reuse train and test
def train(model, optimizer, x_type='x', edge_type='edge_index'):
  criterion = torch.nn.CrossEntropyLoss()
  model.train()
  optimizer.zero_grad()  # Clear gradients.
  out = model(data[x_type], data[edge_type])  # Perform a single forward pass.
  loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  return loss

def test(model, x_type='x', edge_type='edge_index'):
  model.eval()
  out = model(data[x_type], data[edge_type])
  pred = out.argmax(dim=1)  # Use the class with highest probability.
  test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
  test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
  return test_acc

def validation(model, x_type='x', edge_type='edge_index'):
  model.eval()
  out = model(data[x_type], data[edge_type])
  pred = out.argmax(dim=1)  # Use the class with highest probability.
  val_correct = pred[data.val_mask] == data.y[data.val_mask]  # Check against ground-truth labels.
  val_acc = int(val_correct.sum()) / int(data.val_mask.sum())  # Derive ratio of correct predictions.
  return val_acc

# GNN Parameter tuning experiment

In [9]:
def gnn_parameter_tuning_experiment(dataset_name, search_space_params, data, evaltype='valid', x_type='x', edge_type='edge_index', repeat_num=1, print_updates=False):
  '''
  Assumes the noisy data is already created and inside the data object (so that we can use same data sample for different models to compare)
  '''

  exp_count = np.prod([len(vals) for vals in list(search_space_params.values())])
  max_val_acc_so_far = 0
  count=0
  res = []
  num_epochs = search_space_params['epochs'][0]
  in_chs = search_space_params['in_channels'][0]
  out_chs = search_space_params['out_channels'][0]
  print(f'Total experiments to run: {exp_count}')
  for hid_ch in search_space_params['hidden_channels']:
    for num_layer in search_space_params['num_layers']:
      for dropout in search_space_params['dropout']:
        for lr in search_space_params['lrs']:
          for aggr in search_space_params['aggr']:
            for weight_decay in search_space_params['weight_decay']:
              count+=1
              val_accs = []
              for exp_num in range(1, repeat_num+1): # we will repeat experiment repeat many times, to increase results reliability
                  model_params = {'in_channels':in_chs, 'out_channels':out_chs,'hidden_channels':hid_ch, 
                                  'num_layers':num_layer, 'dropout':dropout, 'aggr' :aggr}
                  #print(model_params)
                  gnn_model = GCN(**model_params)
                  optimizer = torch.optim.Adam(gnn_model.parameters(), lr=lr, weight_decay=weight_decay) # parametrize similar to model later TODO

                  for epoch in range(num_epochs):
                    loss = train(gnn_model, optimizer, x_type=x_type, edge_type=edge_type)
                    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

                  # we test our results in original data, no noise added ones 
                  # if you want to test on noised data change them to x_type=x_type and edge_type=ed_type below
                  if evaltype =='valid':
                    val_acc = validation(gnn_model, x_type='x',edge_type='edge_index') 
                  elif evaltype =='test':
                    val_acc = test(gnn_model, x_type='x',edge_type='edge_index') 
                  val_accs.append(val_acc)
              exp_params = model_params
              exp_params['lr']=lr
              exp_params['weight_decay']=weight_decay
              exp_params['epochs']=num_epochs
              mean_val_acc = np.mean(val_accs)
              max_val_acc_so_far = max(max_val_acc_so_far, mean_val_acc )
              if print_updates and count%50==0:
                #print(f'Run exp {count}/{exp_count}: Mean Test Accuracy: {mean_test_acc:.4f}, exp parameters: {exp_params}')
                print(f'Run exp {count}/{exp_count}: Maximum found Avg. Validation Accuracy: {max_val_acc_so_far:.4f}')


              # TODO we can add other metrics such as runtime to log in here later
              exp_res = {'dataset_name': dataset_name,
                        'model_name':'GNN',  'mean_test_accuracy':mean_val_acc, 
                        'exp_params':exp_params, 'tot_experiment':repeat_num,
                          'x_type':x_type, 'edge_type':edge_type} 
              res.append(exp_res)
  print(f'End of experiments..\n  Maximum found Avg. Validation Accuracy: {max_val_acc_so_far:.4f}')
  res_df = pd.DataFrame(res, columns=exp_res.keys())
  return res_df

In [ ]:
search_space_params= {
    'in_channels': [1433],
    'hidden_channels': [5, 15, 30, 50, 100],
    'num_layers': [2, 3, 4],
    'out_channels': [7],
    'dropout' : [0.1, 0.2, 0.3],
    'epochs': [20], 
    ## additinonal message passing parameters
    'aggr' : ["add", "mean", "max"],
    ## optimizer parameters
    'lrs' : [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05],
    'weight_decay' : [1e-6, 5e-4, 1e-4, 1e-3] 
}


In [ ]:
tuning_res_df = gnn_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=search_space_params, 
                                data=data,
                                repeat_num=1, 
                                print_updates=True)

Total experiments to run: 3240
Run exp 50/3240: Maximum found Avg. Validation Accuracy: 0.5980
Run exp 100/3240: Maximum found Avg. Validation Accuracy: 0.6300
Run exp 150/3240: Maximum found Avg. Validation Accuracy: 0.6960
Run exp 200/3240: Maximum found Avg. Validation Accuracy: 0.6960
Run exp 250/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 300/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 350/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 400/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 450/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 500/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 550/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 600/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 650/3240: Maximum found Avg. Validation Accuracy: 0.7220
Run exp 700/3240: Maximum found Avg. Validation Accuracy: 0.7700
Run exp 750/3240: Maximum found Avg. Validation Accuracy: 0.

In [ ]:
tuning_res_df.to_csv('gnn_tuning_res_epoch_20.csv')

In [ ]:
tuning_res_df

,dataset_name,model_name,mean_test_accuracy,exp_params,tot_experiment,x_type,edge_type
0,Cora,GNN,0.391333,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",3,x,edge_index
1,Cora,GNN,0.648667,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",3,x,edge_index
2,Cora,GNN,0.566667,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",3,x,edge_index
3,Cora,GNN,0.688000,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",3,x,edge_index


In [ ]:
tuning_res_df[tuning_res_df.mean_test_accuracy == 0.8100].exp_params.values

array([{'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 3, 'dropout': 0.2, 'aggr': 'add', 'lr': 0.01, 'weight_decay': 1e-06, 'epochs': 20},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 4, 'dropout': 0.1, 'aggr': 'add', 'lr': 0.005, 'weight_decay': 1e-06, 'epochs': 20}],
      dtype=object)

In [ ]:
tuning_res_df = tuning_res_df.sort_values(by='mean_test_accuracy', ascending=False)
tuning_res_df.head(10)['exp_params'].values

array([{'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 3, 'dropout': 0.2, 'aggr': 'add', 'lr': 0.01, 'weight_decay': 1e-06, 'epochs': 20},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 4, 'dropout': 0.1, 'aggr': 'add', 'lr': 0.005, 'weight_decay': 1e-06, 'epochs': 20},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 3, 'dropout': 0.3, 'aggr': 'add', 'lr': 0.01, 'weight_decay': 1e-06, 'epochs': 20},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 2, 'dropout': 0.1, 'aggr': 'add', 'lr': 0.05, 'weight_decay': 0.0001, 'epochs': 20},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 3, 'dropout': 0.3, 'aggr': 'add', 'lr': 0.01, 'weight_decay': 0.0001, 'epochs': 20},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 50, 'num_layers': 2, 'dropout': 0.3, 'aggr': 'add', 'lr': 0.05, 'weight_decay': 

In [ ]:
best_space_params1= {
    'in_channels': [1433],
    'hidden_channels': [100],
    'num_layers': [3],
    'out_channels': [7],
    'dropout' : [0.2],
    'epochs': [30], 
    ## additinonal message passing parameters
    'aggr' : ["add"],
    ## optimizer parameters
    'lrs' : [0.01],
    'weight_decay' : [1e-6] 
}
best_tuning_res_df1 = gnn_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=best_space_params1, 
                                data=data,
                                evaltype='valid',
                                repeat_num=10, 
                                print_updates=True)

Total experiments to run: 1
End of experiments..
  Maximum found Avg. Validation Accuracy: 0.7742


In [ ]:
best_space_params2= {
    'in_channels': [1433],
    'hidden_channels': [50],
    'num_layers': [2],
    'out_channels': [7],
    'dropout' : [0.1],
    'epochs': [100], 
    ## additinonal message passing parameters
    'aggr' : ["add"],
    ## optimizer parameters
    'lrs' : [0.05],
    'weight_decay' : [0.001] 
}
best_tuning_res_df2 = gnn_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=best_space_params2, 
                                data=data,
                                evaltype='valid',
                                repeat_num=10, 
                                print_updates=True)

Total experiments to run: 1
End of experiments..
  Maximum found Avg. Validation Accuracy: 0.7942


In [ ]:
best_space_params3= {
    'in_channels': [1433],
    'hidden_channels': [30],
    'num_layers': [2],
    'out_channels': [7],
    'dropout' : [0.2],
    'epochs': [20],
    ## additinonal message passing parameters
    'aggr' : ["add"],
    ## optimizer parameters
    'lrs' : [0.05],
    'weight_decay' : [1e-6] 
}
best_tuning_res_df3 = gnn_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=best_space_params3, 
                                data=data,
                                evaltype='valid',
                                repeat_num=10, 
                                print_updates=True)

Total experiments to run: 1
End of experiments..
  Maximum found Avg. Validation Accuracy: 0.7848


In [ ]:
best_space_params4= {
    'in_channels': [1433],
    'hidden_channels': [16],
    'num_layers': [2],
    'out_channels': [7],
    'dropout' : [0.2],
    'epochs': [100], 
    ## additinonal message passing parameters
    'aggr' : ["add"],
    ## optimizer parameters
    'lrs' : [0.01],
    'weight_decay' : [5e-4] 
}
best_tuning_res_df4 = gnn_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=best_space_params4, 
                                data=data,
                                evaltype='valid',
                                repeat_num=10, 
                                print_updates=True)

Total experiments to run: 1
End of experiments..
  Maximum found Avg. Validation Accuracy: 0.7886


In [14]:
# test for the best one 
best_space_params_chosen= {
    'in_channels': [1433],
    'hidden_channels': [30],
    'num_layers': [2],
    'out_channels': [7],
    'dropout' : [0.2],
    'epochs': [25],
    ## additinonal message passing parameters
    'aggr' : ["add"],
    ## optimizer parameters
    'lrs' : [0.05],
    'weight_decay' : [1e-6] 
}
final_best_tuning_res_df = gnn_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=best_space_params_chosen, 
                                data=data,
                                evaltype='test',
                                repeat_num=10, 
                                print_updates=True)

Total experiments to run: 1
End of experiments..
  Maximum found Avg. Validation Accuracy: 0.7941


# MLP Tuning Results

In [ ]:
def mlp_parameter_tuning_experiment(dataset_name, search_space_params, data, evaltype='valid', x_type='x',edge_type='edge_index', repeat_num=1, print_updates=False):
  '''
  Assumes the noisy data is already created and inside the data object (so that we can use same data sample for different models to compare)
  '''

  exp_count = np.prod([len(vals) for vals in list(search_space_params.values())])
  max_val_acc_so_far = 0
  count=0
  res = []
  num_epochs = search_space_params['epochs'][0]
  in_chs = search_space_params['in_channels'][0]
  out_chs = search_space_params['out_channels'][0]
  print(f'Total experiments to run: {exp_count}')
  for hid_ch in search_space_params['hidden_channels']:
    for num_layer in search_space_params['num_layers']:
      for dropout in search_space_params['dropout']:
        for lr in search_space_params['lrs']:
            for weight_decay in search_space_params['weight_decay']:
              count+=1
              val_accs = []
              for exp_num in range(1, repeat_num+1): # we will repeat experiment repeat many times, to increase results reliability
                  model_params = {'in_channels':in_chs, 'out_channels':out_chs,'hidden_channels':hid_ch, 
                                  'num_layers':num_layer, 'dropout':dropout}
                  #print(model_params)
                  mlp_model = MLP(**model_params).to('cuda:0')
                  optimizer = torch.optim.Adam(mlp_model.parameters(), lr=lr, weight_decay=weight_decay) # parametrize similar to model later TODO

                  for epoch in range(num_epochs):
                    loss = train(mlp_model, optimizer, model_name='MLP', x_type=x_type, edge_type='edge_index')
                    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

                  # we test our results in original data, no noise added ones 
                  # if you want to test on noised data change them to x_type=x_type and edge_type=ed_type below
                  if evaltype =='valid':
                    val_acc = validation(mlp_model, model_name='MLP',  x_type='x',edge_type='edge_index') 
                  elif evaltype =='test':
                    val_acc = test(mlp_model, model_name='MLP',  x_type='x', edge_type='edge_index') 
                  val_accs.append(val_acc)
              exp_params = model_params
              exp_params['lr']=lr
              exp_params['weight_decay']=weight_decay
              exp_params['epochs']=num_epochs
              mean_val_acc = np.mean(val_accs)
              max_val_acc_so_far = max(max_val_acc_so_far, mean_val_acc )
              if print_updates and count%50==0:
                #print(f'Run exp {count}/{exp_count}: Mean Test Accuracy: {mean_test_acc:.4f}, exp parameters: {exp_params}')
                print(f'Run exp {count}/{exp_count}: Maximum found Avg. Validation Accuracy: {max_val_acc_so_far:.4f}')


              # TODO we can add other metrics such as runtime to log in here later
              exp_res = {'dataset_name': dataset_name,
                        'model_name':'GNN',  'mean_test_accuracy':mean_val_acc, 
                        'exp_params':exp_params, 'tot_experiment':repeat_num,
                          'x_type':x_type} 
              res.append(exp_res)
  print(f'End of experiments..\n  Maximum found Avg. Validation Accuracy: {max_val_acc_so_far:.4f}')
  res_df = pd.DataFrame(res, columns=exp_res.keys())
  return res_df

In [ ]:
search_space_params= {
    'in_channels': [1433],
    'hidden_channels': [5, 15, 30, 50, 100],
    'num_layers': [2, 3, 4],
    'out_channels': [7],
    'dropout' : [0.1, 0.2, 0.3],
    'epochs': [25], 
    ## optimizer parameters
    'lrs' : [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05],
    'weight_decay' : [1e-6, 5e-4, 1e-4, 1e-3] 
}


In [ ]:
data = data.to('cuda:0')
tuning_res_df_mlp = mlp_parameter_tuning_experiment(dataset_name='Cora', 
                                search_space_params=search_space_params, 
                                data=data,
                                eval_type='valid',
                                repeat_num=10, 
                                print_updates=True)
data = data.to('cpu')

Total experiments to run: 1080
Run exp 50/1080: Maximum found Avg. Validation Accuracy: 0.3420
Run exp 100/1080: Maximum found Avg. Validation Accuracy: 0.3420
Run exp 150/1080: Maximum found Avg. Validation Accuracy: 0.3420
Run exp 200/1080: Maximum found Avg. Validation Accuracy: 0.3420
Run exp 250/1080: Maximum found Avg. Validation Accuracy: 0.4534
Run exp 300/1080: Maximum found Avg. Validation Accuracy: 0.4878
Run exp 350/1080: Maximum found Avg. Validation Accuracy: 0.4878
Run exp 400/1080: Maximum found Avg. Validation Accuracy: 0.4878
Run exp 450/1080: Maximum found Avg. Validation Accuracy: 0.4878
Run exp 500/1080: Maximum found Avg. Validation Accuracy: 0.5198
Run exp 550/1080: Maximum found Avg. Validation Accuracy: 0.5198
Run exp 600/1080: Maximum found Avg. Validation Accuracy: 0.5198
Run exp 650/1080: Maximum found Avg. Validation Accuracy: 0.5198
Run exp 700/1080: Maximum found Avg. Validation Accuracy: 0.5198
Run exp 750/1080: Maximum found Avg. Validation Accuracy: 0.

In [ ]:
tuning_res_df_mlp.to_csv('mlp_tuning_res_epoch_25.csv')

In [ ]:
tuning_res_df_mlp = tuning_res_df_mlp.sort_values(by='mean_test_accuracy', ascending=False)
tuning_res_df_mlp.head(10)['exp_params'].values

array([{'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 2, 'dropout': 0.2, 'lr': 0.05, 'weight_decay': 0.001, 'epochs': 25},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 2, 'dropout': 0.3, 'lr': 0.05, 'weight_decay': 0.0001, 'epochs': 25},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 30, 'num_layers': 2, 'dropout': 0.2, 'lr': 0.05, 'weight_decay': 0.001, 'epochs': 25},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 50, 'num_layers': 2, 'dropout': 0.2, 'lr': 0.05, 'weight_decay': 0.001, 'epochs': 25},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100, 'num_layers': 2, 'dropout': 0.3, 'lr': 0.05, 'weight_decay': 0.0005, 'epochs': 25},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 50, 'num_layers': 2, 'dropout': 0.1, 'lr': 0.05, 'weight_decay': 0.001, 'epochs': 25},
       {'in_channels': 1433, 'out_channels': 7, 'hidden_channels': 100,

In [ ]:
tuning_res_df_mlp.head(10)

,dataset_name,model_name,mean_test_accuracy,exp_params,tot_experiment,x_type
911,Cora,GNN,0.5290,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
934,Cora,GNN,0.5238,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
479,Cora,GNN,0.5198,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
695,Cora,GNN,0.5168,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
933,Cora,GNN,0.5108,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
671,Cora,GNN,0.5090,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
935,Cora,GNN,0.5058,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
669,Cora,GNN,0.5054,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
694,Cora,GNN,0.4990,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
932,Cora,GNN,0.4976,"{'in_channels': 1433, 'out_channels': 7, 'hidd...",10,x
